### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *

import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

2025-01-20 22:47:27.663922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(CVXPY) Jan 20 10:47:29 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 20 10:47:29 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

### Preprocessing
Prepare the features and the label. We have two timeseries: sced (real-time) system lambda and day-ahead (DAP) system lambda. We want to predict the real-time system lambda for certain horizon (e.g. 6 hours ahead = 72 points in 5-mins). DAP are available more than 24 hours ahead.  

In [ ]:
sl = pd.read_csv('dataset/ercot_sl_2019_2023.csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('h').mean()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='h')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)


In [4]:
sl

,system_lambda
2019-01-01 00:00:00,14.013557
2019-01-01 01:00:00,15.067093
2019-01-01 02:00:00,15.575292
2019-01-01 03:00:00,16.118229
2019-01-01 04:00:00,16.033707
...,...
2023-12-31 19:00:00,14.422107
2023-12-31 20:00:00,12.597410
2023-12-31 21:00:00,10.170172
2023-12-31 22:00:00,9.872549


In [ ]:
dap = pd.read_csv('dataset/hourly_ercot_day_ahead_sl_2019_2023.csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='h')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

/var/folders/xn/h6sfk2s926b51wv229z3y0g40000gn/T/ipykernel_59684/711271249.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [6]:
dap

,SystemLambda
2019-01-02 00:00:00,23.9250
2019-01-02 01:00:00,23.3140
2019-01-02 02:00:00,23.3475
2019-01-02 03:00:00,23.0595
2019-01-02 04:00:00,25.2672
...,...
2024-01-01 19:00:00,23.1651
2024-01-01 20:00:00,23.2113
2024-01-01 21:00:00,21.3244
2024-01-01 22:00:00,20.3351


In [ ]:
#prepare the data for training and testing
df = pd.concat([dap, sl], axis=1)
df.columns = ['DAP', 'SCED']
df['DAP'] = df['DAP'].shift(-24)
df.dropna(inplace=True)

## log transform
log_data = df.copy(deep=True)
log_data.loc[:,"SCED"] = np.log(df.loc[:,"SCED"] + 1 - min(df.loc[:,"SCED"]))
log_data.loc[:,"DAP"] = np.log(df.loc[:,"DAP"] + 1 - min(df.loc[:,"DAP"]))

# 3 years training, 1 year validation, 1 year testing
x_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:,:]
x_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:,:]
x_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:,:]

y_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:,-1:]
y_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:,-1:]
y_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:,-1:]

x_train_df_reg.reset_index(drop=True, inplace=True)
x_val_df_reg.reset_index(drop=True, inplace=True)
x_test_df_reg.reset_index(drop=True, inplace=True)

# Standardization
x_mean_reg, x_std_reg = x_train_df_reg.mean(), x_train_df_reg.std()
y_mean_reg, y_std_reg = y_train_df_reg.mean(), y_train_df_reg.std()

x_std_reg = x_std_reg +0.00001

x_train_reg = (x_train_df_reg - x_mean_reg)/x_std_reg
x_val_reg = (x_val_df_reg - x_mean_reg)/x_std_reg
x_test_reg = (x_test_df_reg - x_mean_reg)/x_std_reg

y_train_reg = (y_train_df_reg - y_mean_reg)/y_std_reg
y_val_reg = (y_val_df_reg - y_mean_reg)/y_std_reg
y_test_reg = (y_test_df_reg - y_mean_reg)/y_std_reg

# Shift the data for the lags
n_steps_in = 24
n_steps_out = 24

x_train_lstm = np.array([x_train_reg[i:i+n_steps_in] for i in range(0, x_train_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_train_lstm = np.array([y_train_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_val_lstm = np.array([x_val_reg[i:i+n_steps_in] for i in range(0, x_val_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_val_lstm = np.array([y_val_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_val_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_test_lstm = np.array([x_test_reg[i:i+n_steps_in] for i in range(0, x_test_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_test_lstm = np.array([y_test_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test_reg.shape[0]-n_steps_in-n_steps_out+1)])

print(x_train_lstm.shape,y_train_lstm.shape,x_val_lstm.shape,y_val_lstm.shape, x_test_lstm.shape,y_test_lstm.shape)

(26257, 24, 2) (26257, 24, 1) (8713, 24, 2) (8713, 24, 1) (8713, 24, 2) (8713, 24, 1)


### Model Building

In [8]:
# set hyperparameters
n_neurons  = 64  # number of neurons in the Dense layer
activation     = 'relu' # activation function
learning_rate  = 0.0005
minibatch_size = 64
num_epochs     = 50

# MLP model
lstm_model = Sequential()
lstm_model.add(LSTM(n_neurons,input_shape=(x_train_lstm.shape[1],x_train_lstm.shape[2]),
               return_sequences=True,activation=activation))
lstm_model.add(LSTM(n_neurons,return_sequences=False,
               activation=activation))
lstm_model.add(Dense(n_neurons,activation=activation))
lstm_model.add(Dense(y_train_lstm.shape[-2],activation='linear')) 

lstm_model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

lstm_model.summary()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = lstm_model.fit(x_train_lstm, y_train_lstm, 
                        validation_data = (x_val_lstm, y_val_lstm), 
                        batch_size = minibatch_size,
                        epochs = num_epochs,
                        verbose=1,
                        callbacks=[early_stop],
                        shuffle=False)

# saving trianed model
model_path = os.path.join(cwd,'saved_model')
make_dir(model_path)
lstm_model.save(os.path.join(model_path,'hourly_best_model_2424h_sl_dap.h5'))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 64)            17152     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 24)                1560      
                                                                 
Total params: 55896 (218.34 KB)
Trainable params: 55896 (218.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
411/411 [==============================] - 10s 19ms/step - loss: 0.8919 - val_loss: 0.9434
Epoch 2/50
411/411 [============================

/opt/miniconda3/envs/gnn_plot/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Predict and evaluate

In [9]:
# reload the saved model
model_path = os.path.join(cwd,'saved_model')
lstm_model = load_model(os.path.join(model_path,'hourly_best_model_2424h_sl_dap.h5'))

# Make prediciton
y_test_pred = lstm_model.predict(x_test_lstm)

# Evaluate Prediction
evaluate_prediction(y_test_pred , y_test_lstm[:,:,0], 'lstm')

# Rescale to get values before normalization
y_test_pred_rescale = y_test_pred*y_std_reg.values + y_mean_reg.values
y_test_lstm_rescale = y_test_lstm*y_std_reg.values + y_mean_reg.values

# inverse log to get prices in the actual scale
y_test_pred_invlog = np.exp(y_test_pred_rescale) -1 + min(df.loc[:,"SCED"])
y_test_lstm_invlog = np.exp(y_test_lstm_rescale) -1 + min(df.loc[:,"SCED"])

# revaluation after the rescaling
evaluate_prediction(y_test_pred_invlog , y_test_lstm_invlog[:,:,0], 'lstm')

273/273 [==============================] - 2s 4ms/step
MAE: 0.46
RMSE: 1.01


MAE: 31.24
RMSE: 216.06




(31.239261497941367, 216.05661897984552)

In [10]:
df.loc['2023-01-01 23:00:00':'2023-12-31 17:00:00']

,DAP,SCED
2023-01-01 23:00:00,9.46065,-2.003914
2023-01-02 00:00:00,11.22550,-3.867786
2023-01-02 01:00:00,8.27118,-14.568229
2023-01-02 02:00:00,4.19918,-26.102685
2023-01-02 03:00:00,9.14193,-30.359520
...,...,...
2023-12-31 13:00:00,14.46120,10.941627
2023-12-31 14:00:00,13.86100,9.382677
2023-12-31 15:00:00,13.18240,8.912375
2023-12-31 16:00:00,20.28800,16.529858


In [11]:
y_test_lstm_invlog[:,:,0].shape

(8713, 24)

In [12]:
y_test_lstm_invlog[:,:,0]

array([[ -3.86778555, -14.56822896, -26.10268513, ...,  25.54493244,
         18.34069872,   8.52044203],
       [-14.56822896, -26.10268513, -30.35952012, ...,  18.34069872,
          8.52044203,   1.77465975],
       [-26.10268513, -30.35952012, -27.14641682, ...,   8.52044203,
          1.77465975,  -0.89297167],
       ...,
       [ 12.12683145,   8.70062331,   8.1130834 , ...,  14.42210722,
         12.59741012,  10.17017225],
       [  8.70062331,   8.1130834 ,  10.29718486, ...,  12.59741012,
         10.17017225,   9.87254934],
       [  8.1130834 ,  10.29718486,  12.16238181, ...,  10.17017225,
          9.87254934,  12.59412615]])

In [13]:
# add timestamp to the predictions
sl_actual = pd.DataFrame(y_test_lstm_invlog[:,:,0], index=df.loc['2023-01-01 23:00:00':'2023-12-30 23:00:00'].index)
sl_actual.to_csv('hourly_sl_actual_dap_2424h.csv')
sl_pred = pd.DataFrame(y_test_pred_invlog, index=df.loc['2023-01-01 23:00:00':'2023-12-30 23:00:00'].index)
sl_pred.to_csv('hourly_sl_pred_dap_2424h.csv')

In [14]:
sl_pred

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
2023-01-01 23:00:00,2.126856,8.666754,11.329517,12.714264,11.319819,11.486289,12.913681,11.593845,14.131915,13.829476,...,36.060747,26.396742,28.600594,28.271158,27.774318,19.326419,18.604366,12.992094,13.978566,12.776050
2023-01-02 00:00:00,0.591363,7.853952,10.544964,11.133361,10.363530,11.578529,14.662373,13.583106,16.360290,15.845914,...,37.293211,24.421262,26.148905,25.785228,25.118349,16.379990,15.581505,11.560117,11.842564,11.975950
2023-01-02 01:00:00,-4.151479,4.729720,8.953092,10.066285,8.897918,10.604220,14.372339,12.849580,17.679986,14.550124,...,38.084782,21.515739,22.105050,22.343024,21.209614,11.891415,11.178353,6.366667,6.546866,8.592844
2023-01-02 02:00:00,-12.753338,-1.508725,6.041136,8.734697,6.006187,8.571788,11.596972,8.839861,16.616877,10.161917,...,39.548381,17.909730,16.981060,18.088341,15.126153,5.814236,4.904115,-2.544124,-1.779547,1.807257
2023-01-02 03:00:00,-18.977798,-5.861654,2.020915,5.867189,1.980418,5.738524,10.040397,7.056552,16.080111,7.640730,...,39.224491,13.542282,12.819341,13.283703,9.396812,0.125788,-0.303019,-8.363451,-6.649960,-2.501921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 19:00:00,20.904996,24.358423,19.976832,17.411669,21.711305,25.606985,29.565014,32.304873,29.553538,30.674879,...,21.526506,18.421234,19.464634,18.809027,20.391857,19.023819,18.832435,23.569556,25.009922,24.110423
2023-12-30 20:00:00,20.312217,23.922397,21.232502,18.980177,23.073728,26.250016,28.262933,30.057077,27.241252,28.327104,...,20.681684,18.344319,19.623378,19.124189,20.337611,19.465920,19.028881,22.540954,24.158529,23.361201
2023-12-30 21:00:00,18.130067,22.485881,22.044567,20.503863,23.591327,25.833125,25.858168,26.226970,24.084706,25.095692,...,20.760304,18.863930,19.991069,19.916863,20.367106,19.665795,18.379185,19.980231,21.835531,21.254233
2023-12-30 22:00:00,15.469104,20.708320,22.270969,21.448226,23.433878,24.660388,23.260179,22.427549,20.927380,22.064137,...,21.332054,19.647627,20.361870,20.768857,20.530605,19.609397,17.373721,17.284889,19.237161,18.833907


# Storage Arbitrage
Perform storage arbitrage for WAKEWE_ALL node. Use the predicted system lambda to derive the best decisions to maximize profit.
The test year is 2023, so we need the nodal real-time price for that year.

In [15]:
# Storage parameters
horizon = 24 #in hours

Ts = 1  # hourly time step. If the price data is 5-min; Ts = 1/12. If it is hourly; Ts = 1
hr = int(1 / Ts)  # number of time steps in one hour

# Parameters
E = 1  # Energy rating in MWh
Pr = 1 / 2  # normalized power rating wrt energy rating
P = Pr * Ts  # actual power rating taking time step size into account
eta = 0.9  # efficiency
c = 10  # marginal discharge cost - degradation
initial_soc = 0.5

T = hr * horizon #the horizon as number of steps

In [ ]:
# prepare the nodal real-time price in the required shape 
actual = pd.read_csv('datasset/lmp_WAKEWE_ALL_2023.csv')
actual['timestamp'] = pd.to_datetime(actual['timestamp'])
actual.set_index('timestamp', inplace=True)
actual = actual.resample('h').mean().reset_index()

actual_rolled = pd.DataFrame([pd.Series(actual['price'].iloc[i+1:i+1+T].values) for i in range(len(actual) - T)])
actual_rolled.columns = [f'{i+1}' for i in range(T)]
actual_rolled['timestamp'] = actual['timestamp'].iloc[:len(actual_rolled)].values
actual_rolled = actual_rolled[['timestamp'] + [col for col in actual_rolled.columns if col != 'timestamp']]
actual_rolled = actual_rolled.iloc[:,:T+1]

/var/folders/xn/h6sfk2s926b51wv229z3y0g40000gn/T/ipykernel_59684/3690761476.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  actual['timestamp'] = pd.to_datetime(actual['timestamp'])


In [17]:
sced = pd.read_csv('hourly_sl_pred_dap_2424h.csv') #predicted price
pred_rolled = sced.iloc[:,:T+1]
look_back = 24


In [18]:
result_dfs = []

eS = np.zeros(T)  # initialize the SoC series
profit = np.zeros(T)  # initialize the profit series
revenue = np.zeros(T)  # initialize the revenue series

soc_var = cp.Variable(T, nonneg=True)
chr_var = cp.Variable(T, nonneg=True)
dis_var = cp.Variable(T, nonneg=True)
charge_decision = cp.Variable(T, boolean=True)
discharge_decision = cp.Variable(T, boolean=True)
discharge_active = cp.Variable(T, boolean=True)
M = 1e6  # A large constant
price_param = cp.Parameter(T)
soc_init_param = cp.Parameter()

cons = [
    soc_var[0] - soc_init_param == chr_var[0] * eta - dis_var[0] / eta,
    *[soc_var[i] - soc_var[i-1] == chr_var[i] * eta - dis_var[i] / eta for i in range(1, T)],
    *[chr_var[i] <= P for i in range(T)],
    *[soc_var[i] <= E for i in range(T)],
    *[charge_decision[i] + discharge_decision[i] <= 1 for i in range(T)],
    *[dis_var[i] <= discharge_active[i] * P for i in range(T)],
    *[dis_var[i] >= 0 for i in range(T)],
    *[price_param[i] >= -M * (1 - discharge_active[i]) for i in range(T)],
    *[price_param[i] <= M * discharge_active[i] for i in range(T)]
]

revenue_obj = cp.multiply(price_param, (dis_var - chr_var)) - c * dis_var
problem = cp.Problem(cp.Maximize(cp.sum(revenue_obj)), cons)

for start in tqdm(range(int(len(actual_rolled) - look_back))):
    slice_actual = actual_rolled.iloc[look_back + start, :]
    slice_pred = pred_rolled.iloc[start, :]

    price_param.value = slice_pred.iloc[1:].values.astype(float)
    soc_init_param.value = initial_soc
    problem.solve(solver=cp.GUROBI)
    pS_dis = dis_var.value
    pS_chr = chr_var.value
    eS = soc_var.value
    revenue = slice_actual.iloc[1:].values.astype(float) * (pS_dis - pS_chr)
    profit = revenue - c * pS_dis
    storage_profile_sl = pd.Series({
        'time': slice_actual.iloc[0],
        'actual': slice_actual.iloc[1],
        'pred': slice_pred.iloc[1],
        'discharge': pS_dis[0],
        'charge': pS_chr[0],
        'soc': eS[0],
        'profit': profit[0],
        'revenue': revenue[0]})
    result_dfs.append(storage_profile_sl)
    initial_soc = eS[0]

final_result_lmp = pd.DataFrame(result_dfs)

final_result_lmp.to_csv('new_final_result_pred_sl_dap_'+str(T)+'_lmp_2424h_hourly.csv', index=False)



  0%|          | 0/8712 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-09


100%|██████████| 8712/8712 [01:07<00:00, 128.15it/s]


In [19]:
final_result_lmp

,time,actual,pred,discharge,charge,soc,profit,revenue
0,2023-01-02 00:00:00,-14.565000,2.126856,0.0,0.500000,0.95,7.282500,7.282500
1,2023-01-02 01:00:00,-26.102500,0.591363,0.0,0.055556,1.00,1.450139,1.450139
2,2023-01-02 02:00:00,-30.360833,-4.151479,0.0,0.000000,1.00,-0.000000,-0.000000
3,2023-01-02 03:00:00,-27.146667,-12.753338,0.0,0.000000,1.00,-0.000000,-0.000000
4,2023-01-02 04:00:00,-5.260833,-18.977798,0.0,0.000000,1.00,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...
8707,2023-12-30 19:00:00,18.983333,21.124506,0.0,0.000000,1.00,0.000000,0.000000
8708,2023-12-30 20:00:00,15.567500,20.904996,0.0,0.000000,1.00,0.000000,0.000000
8709,2023-12-30 21:00:00,13.187500,20.312217,0.0,0.000000,1.00,0.000000,0.000000
8710,2023-12-30 22:00:00,8.390833,18.130067,0.0,0.000000,1.00,0.000000,0.000000


In [20]:
final_result_lmp.profit.sum()

30380.530439969138